# **Post-Earthquake Building Damage Prediction with Nepal 2015 Dataset**
[Medium Article](https://medium.com/@durbaafaisal/post-earthquake-building-damage-prediction-with-nepal-2015-dataset-48478f1394c6)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.utils import class_weight
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
!pip install --quiet keras_visualizer
from keras_visualizer import visualizer
from IPython.display import Image

In [ ]:
drive.mount('/content/drive')

In [ ]:
# Load dataset

path = '/path_to_files/'
train_values_path = os.path.join(path, 'train_values.csv')
train_labels_path = os.path.join(path, 'train_labels.csv')
train_values = pd.read_csv(train_values_path, index_col='building_id')
train_labels = pd.read_csv(train_labels_path, index_col='building_id')
df = train_values.join(train_labels)
df.head()

# **Exploratory Data Analysis**

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
#Identifying geographic location class number

print(len(df['geo_level_3_id'].unique()))
print(len(df['geo_level_2_id'].unique()))
print(len(df['geo_level_1_id'].unique()))

In [ ]:
missing_counts = df.isnull().sum()
missing_percentages = (missing_counts / len(df)) * 100

print(missing_counts, missing_percentages)

In [ ]:
# @title Numerical Feature Overview
TARGET_COLUMN = 'damage_grade'

print(f"\n--- Analysis of Target Variable: {TARGET_COLUMN} ---")

print("\nValue Counts:")
print(df[TARGET_COLUMN].value_counts())

print("\nPercentage Distribution:")
print(df[TARGET_COLUMN].value_counts(normalize=True).map('{:.2%}'.format))

# Visualize the distribution
plt.figure(figsize=(8, 5))
sns.countplot(x=TARGET_COLUMN, data=df, palette='viridis', order=df[TARGET_COLUMN].value_counts().index)
plt.title('Distribution of Damage Grades')
plt.xlabel('Damage Grade')
plt.ylabel('Count')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()

# Remove target variable and columns we might treat as categorical (like IDs or binary flags)
exclude = [TARGET_COLUMN]
exclude.extend([col for col in df.columns if col.startswith('geo_level')])
exclude.extend([col for col in df.columns if col.startswith('has_')])

numerical_cols_for_plots = [col for col in numerical_cols if col not in exclude]

print(f"\n--- Plotting Distributions for {len(numerical_cols_for_plots)} Numerical Features ---")
print(f"Features: {numerical_cols_for_plots}")

# Determine grid size for subplots
n_cols_grid = 3
n_rows_grid = (len(numerical_cols_for_plots) - 1) // n_cols_grid + 1

plt.figure(figsize=(n_cols_grid * 5, n_rows_grid * 4)) # Adjust figure size as needed

for i, col in enumerate(numerical_cols_for_plots):
    plt.subplot(n_rows_grid, n_cols_grid, i + 1)
    sns.histplot(df[col], kde=True, bins=30) # Add kernel density estimate
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
# Use the same numerical_cols_for_plots as above
plt.figure(figsize=(n_cols_grid * 5, n_rows_grid * 4.5)) # Adjust figure size

for i, col in enumerate(numerical_cols_for_plots):
    plt.subplot(n_rows_grid, n_cols_grid, i + 1)
    sns.boxplot(x=TARGET_COLUMN, y=col, data=df, palette='viridis')
    plt.title(f'{col} vs. {TARGET_COLUMN}')
    plt.xlabel('Damage Grade')
    plt.ylabel(col)
    plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

In [ ]:
cols_for_corr = numerical_cols_for_plots + [TARGET_COLUMN]

correlation_matrix = df[cols_for_corr].corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Correlation Matrix (Numerical Features & Damage Grade)')
plt.show()

# Display correlations specifically with the target variable
print(f"\n--- Correlations with Target Variable ({TARGET_COLUMN}) ---")
target_correlations = correlation_matrix[TARGET_COLUMN].drop(TARGET_COLUMN).sort_values(ascending=False)
print(target_correlations)

In [ ]:
# @title Categorical Feature Overview
print("\n--- Analyzing Categorical Features ---")

# Identify categorical columns (object dtype)
categorical_cols_obj = df.select_dtypes(include='object').columns.tolist()
print(f"Object type columns ({len(categorical_cols_obj)}): {categorical_cols_obj}")

# Also consider 'geo_level_id's as categorical, and binary flags
geo_cols = [col for col in df.columns if col.startswith('geo_level')]
binary_flag_cols = [col for col in df.columns if col.startswith('has_')] # Includes superstructure and secondary use for now

# Combine for a broader list (excluding target)
all_potential_categorical = sorted(list(set(categorical_cols_obj + geo_cols + binary_flag_cols)))
all_potential_categorical = [col for col in all_potential_categorical if col != TARGET_COLUMN]

print(f"\n--- Unique Value Counts for Potential Categorical Features ---")
# Calculate and display unique counts for each
unique_counts = {col: df[col].nunique() for col in all_potential_categorical if col in df.columns}

# Sort by number of unique values for clarity
unique_counts_sorted = dict(sorted(unique_counts.items(), key=lambda item: item[1]))

for col, count in unique_counts_sorted.items():
    print(f"- {col}: {count} unique values")

# Define key low-cardinality categoricals for detailed plots (exclude binary flags for now)
key_categoricals = [
    'land_surface_condition', 'foundation_type', 'roof_type',
    'ground_floor_type', 'other_floor_type', 'position',
    'plan_configuration', 'legal_ownership_status'
]
# Filter to only those present in the current df
key_categoricals = [col for col in key_categoricals if col in df.columns]

In [ ]:

n_key_cats = len(key_categoricals)
n_cols_grid_cat = 2
n_rows_grid_cat = (n_key_cats - 1) // n_cols_grid_cat + 1

plt.figure(figsize=(n_cols_grid_cat * 7, n_rows_grid_cat * 5)) # Adjust figure size

for i, col in enumerate(key_categoricals):
    plt.subplot(n_rows_grid_cat, n_cols_grid_cat, i + 1)
    # Order bars by the total frequency of the category
    order = df[col].value_counts().index
    sns.countplot(data=df, x=col, hue=TARGET_COLUMN, order=order, palette='viridis')
    plt.title(f'{TARGET_COLUMN} Distribution by {col}')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right') # Rotate labels if needed
    plt.legend(title='Damage Grade')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


# It's hard to plot all flags. Let's look at a few important ones vs damage.
superstructure_flags_to_check = [
    'has_superstructure_rc_engineered',
    'has_superstructure_rc_non_engineered',
    'has_superstructure_mud_mortar_stone',
    'has_superstructure_timber',
    'has_superstructure_adobe_mud'
]


print(f"\n--- Plotting Selected Superstructure Flags vs. {TARGET_COLUMN} ---")
n_flags = len(superstructure_flags_to_check)
plt.figure(figsize=(n_flags * 4, 16))

for i, flag in enumerate(superstructure_flags_to_check):
    plt.subplot(1, n_flags, i + 1)
    # Ensure the flag column is treated as categorical for hue grouping
    sns.countplot(data=df, x=flag, hue=TARGET_COLUMN, palette='viridis')
    plt.title(f'Damage by {flag}')
    plt.xlabel(f'Has {flag.split("_")[-1]}? (0=No, 1=Yes)')
    plt.ylabel('Count')
    plt.legend(title='Grade', loc='upper center') # Adjust legend position
    plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


# --- Analysis for Geo Level 1 ID ---
print(f"\n--- Plotting {TARGET_COLUMN} Distribution by geo_level_1_id ---")
plt.figure(figsize=(14, 6))
order_geo1 = df['geo_level_1_id'].value_counts().index # Order by frequency
sns.countplot(data=df, x='geo_level_1_id', hue=TARGET_COLUMN, order=order_geo1, palette='viridis')
plt.title(f'{TARGET_COLUMN} Distribution by geo_level_1_id')
plt.xlabel('Geo Level 1 ID')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Damage Grade')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
print(f"\n--- Plotting Selected Superstructure Flags vs. {TARGET_COLUMN} ---")
n_flags = len(superstructure_flags_to_check)
n_cols = 2  # Number of columns in the grid (set to 2)
n_rows = (n_flags + n_cols - 1) // n_cols  # Number of rows (calculated)
plt.figure(figsize=(n_cols * 4, n_rows * 4))  # Adjust figure size

for i, flag in enumerate(superstructure_flags_to_check):
    plt.subplot(n_rows, n_cols, i + 1)
    # Ensure the flag column is treated as categorical for hue grouping
    sns.countplot(data=df, x=flag, hue=TARGET_COLUMN, palette='viridis')
    plt.title(f'Damage by {flag}')
    plt.xlabel(f'Has {flag.split("_")[-1]}? (0=No, 1=Yes)')
    plt.ylabel('Count')
    plt.legend(title='Grade', loc='upper center')
    plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# **Preprocessing**

In [ ]:
# Dropping low impact / problematic features based on EDA
irrelevant = [col for col in df.columns if col.startswith('has_secondary_')]
irrelevant.append('count_families')
irrelevant.append('legal_ownership_status')
irrelevant.append('geo_level_3_id')
irrelevant.append('position')

df.drop(columns=irrelevant, inplace=True)
df

In [ ]:
# Splitting the dataset
y = df[TARGET_COLUMN]
X = df.drop(columns = [TARGET_COLUMN])

y = y-1 # adjusting target / damage grade to be 0 indexed

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y,
    test_size=0.15,
    random_state=13,
    stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=0.1765,
    random_state=17,
    stratify=y_train_val
)

In [ ]:
# columns for one-hot encoding
ohe_cols = sorted(list(set(
    X_train.select_dtypes(include='object').columns.tolist()
    + ['geo_level_1_id' , 'geo_level_2_id'])))

# Binary Passthrough Columns:
bin_cols = sorted([
    col for col in X_train.columns if col.startswith('has_superstructure_')
])

# Numerical Columns to Scale:
num_cols = sorted([
    col for col in X_train.columns if col not in ohe_cols and col not in bin_cols
])

In [ ]:
scaler = StandardScaler()
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore', dtype=np.int8)
scaler.fit(X_train[num_cols])
encoder.fit(X_train[ohe_cols])
ohe_names = encoder.get_feature_names_out(ohe_cols)

In [ ]:
# Transforming Numerical Features
X_train_num = scaler.transform(X_train[num_cols]).astype(np.float32)
X_val_num = scaler.transform(X_val[num_cols]).astype(np.float32)
X_test_num = scaler.transform(X_test[num_cols]).astype(np.float32)

In [ ]:
# Transforming Categorical Features
X_train_ohe = encoder.transform(X_train[ohe_cols]) # dtype=int8 set during init
X_val_ohe = encoder.transform(X_val[ohe_cols])
X_test_ohe = encoder.transform(X_test[ohe_cols])

In [ ]:
# Extracting Passthrough Features
X_train_bin = X_train[bin_cols].to_numpy(dtype=np.int8)
X_val_bin = X_val[bin_cols].to_numpy(dtype=np.int8)
X_test_bin = X_test[bin_cols].to_numpy(dtype=np.int8)

In [ ]:
# Combining features into final numpy arrays
X_train_final = np.hstack([X_train_num, X_train_bin, X_train_ohe])
X_val_final = np.hstack([X_val_num, X_val_bin, X_val_ohe])
X_test_final = np.hstack([X_test_num, X_test_bin, X_test_ohe])

print(f"Final combined array shape (Train): {X_train_final.shape}")
print(f"Final combined array shape (Validation):   {X_val_final.shape}")
print(f"Final combined array shape (Test):  {X_test_final.shape}")


# Deleting large intermediate arrays
import gc
del X_train_num, X_val_num, X_test_num
del X_train_ohe, X_val_ohe, X_test_ohe
del X_train_bin, X_val_bin, X_test_bin
gc.collect()

In [ ]:
# Transforming y as well to numpy for efficiency
y_train_final = y_train.to_numpy()
y_val_final = y_val.to_numpy()
y_test_final = y_test.to_numpy()

print(f"Final target array shape (Train): {y_train_final.shape}")
print(f"Final target array shape (Val):   {y_val_final.shape}")
print(f"Final target array shape (Test):  {y_test_final.shape}")
print(f"Target arrays dtype: {y_train_final.dtype}")
print(f"Unique values in y_train_final: {np.unique(y_train_final)}")

# **Model Development & Training the Model**

In [ ]:
# Clearing any previous Keras session state
keras.backend.clear_session()

model = Sequential(name="EarthquakeDamage_MLP_v1")

model.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))

model.add(Dense(128, activation='relu', name='Hidden_Layer_1'))
model.add(Dropout(rate=0.3, name='Dropout_1'))
model.add(Dense(64, activation='relu', name='Hidden_Layer_2'))
model.add(Dropout(rate=0.3, name='Dropout_2'))

# Output Layer
model.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

# Compiling the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

In [ ]:
# Early stopping function settings for efficiency
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)

In [ ]:
# Training the model
history = model.fit(
    X_train_final,
    y_train_final,
    batch_size=128,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[early_stopping],
    verbose=1
)

In [ ]:
# Visualization
history_df = pd.DataFrame(history.history)

plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history_df['loss'], label='Training Loss')
plt.plot(history_df['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_df['accuracy'], label='Training Accuracy')
plt.plot(history_df['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch = history_df['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch + 1}")
print(f" - Best Validation Loss: {history_df.loc[best_epoch, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_df.loc[best_epoch, 'val_accuracy']:.4f}")

# **Testing and Evaluation**

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test_final, y_test_final, verbose=1)
print(f"\nTest Set Evaluation:")
print(f" - Test Loss: {test_loss:.4f}")
print(f" - Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

y_pred_probs = model.predict(X_test_final)
y_pred_classes = np.argmax(y_pred_probs, axis=1) # selecting the class with highes probability

manual_accuracy = accuracy_score(y_test_final, y_pred_classes) # manually calculating accuracy
print(f"\nManual Accuracy Check: {manual_accuracy:.4f} ({manual_accuracy*100:.2f}%)")

In [ ]:
# Detailed Classification Metrics
target_names = ['Grade 1', 'Grade 2', 'Grade 3']

print(classification_report(y_test_final, y_pred_classes, target_names=target_names))

cm = confusion_matrix(y_test_final, y_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

In [ ]:
row_sums = cm.sum(axis=1)

cm_normalized = cm.astype('float') / row_sums[:, np.newaxis]

plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names if 'target_names' in locals() else sorted(np.unique(y_test_final)),
            yticklabels=target_names if 'target_names' in locals() else sorted(np.unique(y_test_final)))
plt.title('Normalized Confusion Matrix (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

# **Refining the Model**

The number of trials mentioned in the article doesn't match the number here because some sections were re-used multiple times while refinement.

## **Trial 1 Balancing Weights**

In [ ]:
# Clear any previous Keras session state
keras.backend.clear_session()

# Build the Sequential Model
model_weighted = Sequential(name="EarthquakeDamage_MLP_Weighted")
model_weighted.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))

model_weighted.add(Dense(128, activation='relu', name='Hidden_Layer_1'))
model_weighted.add(Dropout(0.3, name='Dropout_1'))
model_weighted.add(Dense(64, activation='relu', name='Hidden_Layer_2'))
model_weighted.add(Dropout(0.3, name='Dropout_2'))

# Output Layer
model_weighted.add(Dense(3, activation='softmax', name='Output_Layer'))

# Compiling the Model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_weighted.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_weighted.summary()

In [ ]:
unique_classes = np.unique(y_train_final)

weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=unique_classes,
    y=y_train_final
)

class_weights_dict = dict(zip(unique_classes, weights))

# Early stopping function
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)

# Training the model
history_weighted = model_weighted.fit(
    X_train_final,
    y_train_final,
    batch_size=128,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[early_stopping],
    class_weight=class_weights_dict,
    verbose=1
)

In [ ]:
class_weights_dict

In [ ]:
history_df_w = pd.DataFrame(history_weighted.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_df_w['loss'], label='Training Loss')
plt.plot(history_df_w['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_df_w['accuracy'], label='Training Accuracy')
plt.plot(history_df_w['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_w = history_df_w['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_w + 1}")
print(f" - Best Validation Loss: {history_df_w.loc[best_epoch_w, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_df_w.loc[best_epoch_w, 'val_accuracy']:.4f}")


In [ ]:
test_loss_w, test_accuracy_w = model_weighted.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = model_weighted.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix

cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()


Accuracy: Dropped significantly to 67.73% (from 73.45%).
Loss: Increased to 0.6813 (from 0.5880).
Conclusion: Applying balanced class weights decreased the overall accuracy and increased the overall loss on the test set.

## **Trial 2: Balancing weights manually**


In [ ]:
unique_classes = np.unique(y_train_final)
balanced_weights_array = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=unique_classes,
    y=y_train_final
)

balanced_weights_dict = dict(zip(unique_classes, balanced_weights_array))


boost_factor = 0.7 #


# 5. Create Manual Weight Dictionary: Iterate through each class to calculate its weight.
manual_weights_dict = {}

for class_index in unique_classes:

    if class_index == 1: # majority class
        manual_weights_dict[class_index] = 1.0
        print(f" - Class {class_index} (Majority): Weight = 1.0")
    else:
        balanced_weight = balanced_weights_dict.get(class_index, 1.0)
        adjusted_weight = (balanced_weight - 1)*boost_factor + 1
        manual_weights_dict[class_index] = adjusted_weight
        print(f" - Class {class_index} (Minority): Balanced Weight={balanced_weight:.4f}, Adjusted Weight = {adjusted_weight:.4f}")

In [ ]:
manual_weights_dict = {0:2.9,1:1,2:1.3}

In [ ]:
# Rebuilding the model
keras.backend.clear_session()

model_manual_weights = Sequential(name="EarthquakeDamage_MLP_ManualWeight")
model_manual_weights.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))
model_manual_weights.add(Dense(128, activation='relu', name='Hidden_Layer_1'))
model_manual_weights.add(Dropout(0.3, name='Dropout_1'))
model_manual_weights.add(Dense(64, activation='relu', name='Hidden_Layer_2'))
model_manual_weights.add(Dropout(0.3, name='Dropout_2'))
model_manual_weights.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_manual_weights.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_manual_weights.summary()

In [ ]:
# Training the model with manually weighted balance factors
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)


history_manual_weights = model_manual_weights.fit(
    X_train_final,
    y_train_final,
    batch_size=128,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[early_stopping],
    class_weight=manual_weights_dict,
    verbose=1
)

In [ ]:
history_df_mw = pd.DataFrame(history_manual_weights.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_df_mw['loss'], label='Training Loss')
plt.plot(history_df_mw['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_df_mw['accuracy'], label='Training Accuracy')
plt.plot(history_df_mw['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_mw = history_df_mw['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_mw + 1}")
print(f" - Best Validation Loss: {history_df_mw.loc[best_epoch_mw, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_df_mw.loc[best_epoch_mw, 'val_accuracy']:.4f}")


In [ ]:
test_loss_w, test_accuracy_w = model_manual_weights.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = model_manual_weights.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix
cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()


## **Trial 3: Hyperparameter Tuning**

In [ ]:
# Rebuilding the model
keras.backend.clear_session()

model_hyper = Sequential(name="EarthquakeDamage_MLP_ManualWeight")
model_hyper.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))
model_hyper.add(Dense(128, activation='relu', name='Hidden_Layer_1'))
model_hyper.add(Dropout(0.3, name='Dropout_1'))
model_hyper.add(Dense(64, activation='relu', name='Hidden_Layer_2'))
model_hyper.add(Dropout(0.3, name='Dropout_2'))
model_hyper.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_hyper.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_hyper.summary()

In [ ]:
# Training the model with manually weighted balance factors
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=6,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)


history_hyper = model_hyper.fit(
    X_train_final,
    y_train_final,
    batch_size=256,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[early_stopping],
    class_weight=manual_weights_dict,
    verbose=1
)

In [ ]:
history_hyper = pd.DataFrame(history_hyper.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_hyper['loss'], label='Training Loss')
plt.plot(history_hyper['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_hyper['accuracy'], label='Training Accuracy')
plt.plot(history_hyper['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_hyper = history_hyper['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_hyper + 1}")
print(f" - Best Validation Loss: {history_hyper.loc[best_epoch_hyper, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_hyper.loc[best_epoch_hyper, 'val_accuracy']:.4f}")


In [ ]:
test_loss_w, test_accuracy_w = model_hyper.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = model_hyper.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix
cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()


## **Trial 4: Structure of Hidden Layers**

In [ ]:
keras.backend.clear_session()

model_structure = Sequential(name="EarthquakeDamage_MLP_Structure")
model_structure.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_1'))
model_structure.add(Dropout(0.3, name='Dropout_1'))
model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_2'))
model_structure.add(Dropout(0.3, name='Dropout_2'))
model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_3'))
model_structure.add(Dropout(0.3, name='Dropout_3'))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_4'))
model_structure.add(Dropout(0.3, name='Dropout_4'))
model_structure.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_structure.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_structure.summary()

In [ ]:
visualizer(model_structure, file_name="visual", file_format='png')
Image("visual.png")

In [ ]:
structure_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)


# Train the final model
history_structure = model_structure.fit(
    X_train_final,
    y_train_final,
    batch_size=256,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[structure_early_stopping],
    class_weight=manual_weights_dict,
    verbose=1
)

In [ ]:
history_structure = pd.DataFrame(history_structure.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_structure['loss'], label='Training Loss')
plt.plot(history_structure['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_structure['accuracy'], label='Training Accuracy')
plt.plot(history_structure['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_structure = history_structure['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_structure + 1}")
print(f" - Best Validation Loss: {history_structure.loc[best_epoch_structure, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_structure.loc[best_epoch_structure, 'val_accuracy']:.4f}")


In [ ]:
test_loss_w, test_accuracy_w = model_structure.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = model_structure.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix
cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

## **Trial 5: Structure of Droput Layers**

In [ ]:
keras.backend.clear_session()

model_structure = Sequential(name="EarthquakeDamage_MLP_Structure")
model_structure.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_1'))
model_structure.add(Dropout(0.3, name='Dropout_1'))
model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_2'))

model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_3'))
model_structure.add(Dropout(0.3, name='Dropout_3'))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_4'))
model_structure.add(Dropout(0.3, name='Dropout_4'))
model_structure.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_structure.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_structure.summary()

structure_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)


# Train the final model
history_structure = model_structure.fit(
    X_train_final,
    y_train_final,
    batch_size=256,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[structure_early_stopping],
    class_weight=manual_weights_dict,
    verbose=1
)

history_structure = pd.DataFrame(history_structure.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_structure['loss'], label='Training Loss')
plt.plot(history_structure['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_structure['accuracy'], label='Training Accuracy')
plt.plot(history_structure['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_structure = history_structure['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_structure + 1}")
print(f" - Best Validation Loss: {history_structure.loc[best_epoch_structure, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_structure.loc[best_epoch_structure, 'val_accuracy']:.4f}")


test_loss_w, test_accuracy_w = model_structure.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = model_structure.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix
cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

In [ ]:
visualizer(model_structure, file_name="visual", file_format='png')
Image("visual.png")

## **Trial 6: Structure Dropout Layers**

In [ ]:
keras.backend.clear_session()

model_structure = Sequential(name="EarthquakeDamage_MLP_Structure")
model_structure.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_1'))
model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_2'))
model_structure.add(Dropout(0.3, name='Dropout_2'))
model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_3'))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_4'))
model_structure.add(Dropout(0.3, name='Dropout_4'))
model_structure.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_structure.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_structure.summary()

structure_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)


# Train the final model
history_structure = model_structure.fit(
    X_train_final,
    y_train_final,
    batch_size=256,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[structure_early_stopping],
    class_weight=manual_weights_dict,
    verbose=1
)

history_structure = pd.DataFrame(history_structure.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_structure['loss'], label='Training Loss')
plt.plot(history_structure['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_structure['accuracy'], label='Training Accuracy')
plt.plot(history_structure['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_structure = history_structure['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_structure + 1}")
print(f" - Best Validation Loss: {history_structure.loc[best_epoch_structure, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_structure.loc[best_epoch_structure, 'val_accuracy']:.4f}")


test_loss_w, test_accuracy_w = model_structure.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = model_structure.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix
cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

In [ ]:
visualizer(model_structure, file_name="visual", file_format='png')
Image("visual.png")

## **Trial 7: Structure - Dropout Layers**

In [ ]:
keras.backend.clear_session()

model_structure = Sequential(name="EarthquakeDamage_MLP_Structure")
model_structure.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_1'))
model_structure.add(Dropout(0.3, name='Dropout_1'))
model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_2'))

model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_3'))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_4'))
model_structure.add(Dropout(0.3, name='Dropout_4'))
model_structure.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_structure.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_structure.summary()

structure_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)


# Train the final model
history_structure = model_structure.fit(
    X_train_final,
    y_train_final,
    batch_size=256,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[structure_early_stopping],
    class_weight=manual_weights_dict,
    verbose=1
)

history_structure = pd.DataFrame(history_structure.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_structure['loss'], label='Training Loss')
plt.plot(history_structure['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_structure['accuracy'], label='Training Accuracy')
plt.plot(history_structure['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_structure = history_structure['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_structure + 1}")
print(f" - Best Validation Loss: {history_structure.loc[best_epoch_structure, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_structure.loc[best_epoch_structure, 'val_accuracy']:.4f}")


test_loss_w, test_accuracy_w = model_structure.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = model_structure.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix
cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

In [ ]:
visualizer(model_structure, file_name="visual", file_format='png')
Image("visual.png")

The best structure seems to leave the dropout layers in the middle.

## **Trial 8: Hidden Layer Structure**
Increasing Complexity

In [ ]:
keras.backend.clear_session()

model_structure = Sequential(name="EarthquakeDamage_MLP_Structure")
model_structure.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_1'))
model_structure.add(Dropout(0.3, name='Dropout_1'))
model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_2'))
model_structure.add(Dropout(0.3, name='Dropout_2'))
model_structure.add(Dense(256, activation='relu', name='Hidden_Layer_3'))
model_structure.add(Dropout(0.3, name='Dropout_3'))
model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_4'))
model_structure.add(Dropout(0.3, name='Dropout_4'))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_5'))
model_structure.add(Dropout(0.3, name='Dropout_5'))
model_structure.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_structure.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_structure.summary()

structure_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)


# Train the final model
history_structure = model_structure.fit(
    X_train_final,
    y_train_final,
    batch_size=256,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[structure_early_stopping],
    class_weight=manual_weights_dict,
    verbose=1
)

history_structure = pd.DataFrame(history_structure.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_structure['loss'], label='Training Loss')
plt.plot(history_structure['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_structure['accuracy'], label='Training Accuracy')
plt.plot(history_structure['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_structure = history_structure['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_structure + 1}")
print(f" - Best Validation Loss: {history_structure.loc[best_epoch_structure, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_structure.loc[best_epoch_structure, 'val_accuracy']:.4f}")


test_loss_w, test_accuracy_w = model_structure.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = model_structure.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix
cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

In [ ]:
visualizer(model_structure, file_name="visual", file_format='png')
Image("visual.png")

## **Trial 9: Structure - Hidden and Droupout Layers**

In [ ]:
keras.backend.clear_session()

model_structure = Sequential(name="EarthquakeDamage_MLP_Structure")
model_structure.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_1'))
model_structure.add(Dropout(0.3, name='Dropout_1'))
model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_2'))

model_structure.add(Dense(256, activation='relu', name='Hidden_Layer_3'))

model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_4'))
model_structure.add(Dropout(0.3, name='Dropout_4'))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_5'))
model_structure.add(Dropout(0.3, name='Dropout_5'))
model_structure.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_structure.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_structure.summary()

structure_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)


# Train the final model
history_structure = model_structure.fit(
    X_train_final,
    y_train_final,
    batch_size=256,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[structure_early_stopping],
    class_weight=manual_weights_dict,
    verbose=1
)

history_structure = pd.DataFrame(history_structure.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_structure['loss'], label='Training Loss')
plt.plot(history_structure['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_structure['accuracy'], label='Training Accuracy')
plt.plot(history_structure['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_structure = history_structure['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_structure + 1}")
print(f" - Best Validation Loss: {history_structure.loc[best_epoch_structure, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_structure.loc[best_epoch_structure, 'val_accuracy']:.4f}")


test_loss_w, test_accuracy_w = model_structure.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = model_structure.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix
cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

In [ ]:
visualizer(model_structure, file_name="visual", file_format='png')
Image("visual.png")

## **Trial 10: Structure - Hidden Layers**

In [ ]:
keras.backend.clear_session()

model_structure = Sequential(name="EarthquakeDamage_MLP_Structure")
model_structure.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))
model_structure.add(Dense(32, activation='relu', name='Hidden_Layer_1'))
model_structure.add(Dropout(0.3, name='Dropout_1'))
model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_2'))

model_structure.add(Dense(128, activation='relu', name='Hidden_Layer_3'))

model_structure.add(Dense(64, activation='relu', name='Hidden_Layer_4'))
model_structure.add(Dropout(0.3, name='Dropout_4'))
model_structure.add(Dense(32, activation='relu', name='Hidden_Layer_5'))
model_structure.add(Dropout(0.3, name='Dropout_5'))
model_structure.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_structure.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model_structure.summary()

structure_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)


# Train the final model
history_structure = model_structure.fit(
    X_train_final,
    y_train_final,
    batch_size=256,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[structure_early_stopping],
    class_weight=manual_weights_dict,
    verbose=1
)

history_structure = pd.DataFrame(history_structure.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_structure['loss'], label='Training Loss')
plt.plot(history_structure['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_structure['accuracy'], label='Training Accuracy')
plt.plot(history_structure['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_structure = history_structure['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_structure + 1}")
print(f" - Best Validation Loss: {history_structure.loc[best_epoch_structure, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_structure.loc[best_epoch_structure, 'val_accuracy']:.4f}")


test_loss_w, test_accuracy_w = model_structure.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = model_structure.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix
cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

# **Best Model**

In [ ]:
keras.backend.clear_session()

final_model = Sequential(name="EarthquakeDamage_MLP_ManualWeight")
final_model.add(Input(shape=(X_train_final.shape[1],), name="Input_Layer"))
final_model.add(Dense(128, activation='relu', name='Hidden_Layer_1'))
final_model.add(Dropout(0.3, name='Dropout_1'))
final_model.add(Dense(64, activation='relu', name='Hidden_Layer_2'))
final_model.add(Dropout(0.3, name='Dropout_2'))
final_model.add(Dense(len(np.unique(y_train_final)), activation='softmax', name='Output_Layer'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
final_model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

final_model.summary()

In [ ]:
visualizer(final_model, file_name="visual", file_format='png')
Image("visual.png")

In [ ]:
final_early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1,
    mode='min'
)


# Train the final model
history_final = final_model.fit(
    X_train_final,
    y_train_final,
    batch_size=256,
    epochs=100,
    validation_data=(X_val_final, y_val_final),
    callbacks=[final_early_stopping],
    class_weight=manual_weights_dict,
    verbose=1
)


In [ ]:
history_final = pd.DataFrame(history_final.history)

plt.figure(figsize=(12, 5))

#  Loss
plt.subplot(1, 2, 1)
plt.plot(history_final['loss'], label='Training Loss')
plt.plot(history_final['val_loss'], label='Validation Loss')
plt.title('Weighted Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (Sparse Categorical Crossentropy)')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

# Pot Accuracy
plt.subplot(1, 2, 2)
plt.plot(history_final['accuracy'], label='Training Accuracy')
plt.plot(history_final['val_accuracy'], label='Validation Accuracy')
plt.title('Weighted Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()

# Find epoch with best validation loss
best_epoch_final = history_final['val_loss'].idxmin()
print(f"\nBest Validation Loss achieved at epoch {best_epoch_final + 1}")
print(f" - Best Validation Loss: {history_final.loc[best_epoch_final, 'val_loss']:.4f}")
print(f" - Validation Accuracy at Best Epoch: {history_final.loc[best_epoch_final, 'val_accuracy']:.4f}")


In [ ]:
test_loss_w, test_accuracy_w = final_model.evaluate(X_test_final, y_test_final, verbose=0)

print(f"\nWeighted Test Set Evaluation:")
print(f" - Test Loss: {test_loss_w:.4f}")
print(f" - Test Accuracy: {test_accuracy_w:.4f} ({test_accuracy_w*100:.2f}%)")

y_pred_probs_w = final_model.predict(X_test_final)
y_pred_classes_w = np.argmax(y_pred_probs_w, axis=1)


target_names = ['Grade 1', 'Grade 2', 'Grade 3']
print(classification_report(y_test_final, y_pred_classes_w, target_names=target_names))

# Confusion Matrix
cm_w = confusion_matrix(y_test_final, y_pred_classes_w)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_w, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix - Weighted Model Test Set')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()

row_sums_w = cm_w.sum(axis=1)
cm_normalized_w = cm_w.astype('float') / row_sums_w[:, np.newaxis]
plt.figure(figsize=(8, 6))
sns.heatmap(cm_normalized_w, annot=True, fmt='.2%', cmap='Blues', # Format as percentage
            xticklabels=target_names, yticklabels=target_names)
plt.title('Normalized Confusion Matrix - Weighted Model (% of Actual Class)')
plt.ylabel('Actual Class')
plt.xlabel('Predicted Class')
plt.show()